Adverse media:
===
Extracting:
1. name from text


In [ ]:
!pip install spacy --upgrade

In [ ]:
!python -m spacy download en_core_web_trf

In [ ]:
!pip install Scrapy --upgrade

In [ ]:
!pip install unidecode

--------------- Restart Runtime ----------------

In [ ]:
!scrapy startproject express

In [31]:
!ls

express  express_items.csv  indian_express.json  scrapy.cfg  websites.csv


In [2]:
%cd /content/
!ls express/express/spiders

/content
__init__.py  news_spider.py  __pycache__


In [3]:
%%writefile /content/express/express/items.py
# Define here the models for your scraped items
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/items.html

import scrapy


class ExpressItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    keyword = scrapy.Field()
    hdfcpresent = scrapy.Field()
    text = scrapy.Field()
    name = scrapy.Field()
    org = scrapy.Field()
    loc = scrapy.Field()
    date = scrapy.Field()
    sources = scrapy.Field()
    # headline = scrapy.Field()
    # url = scrapy.Field()
    # pass


Overwriting /content/express/express/items.py


In [4]:
%%writefile /content/express/express/pipelines.py
# Define your item pipelines here
#
# Don't forget to add your pipeline to the ITEM_PIPELINES setting
# See: https://docs.scrapy.org/en/latest/topics/item-pipeline.html


# useful for handling different item types with a single interface
# from itemadapter import ItemAdapter


# class NewsPipeline:
    # def process_item(self, item, spider):
        # return item

from scrapy import signals
from scrapy.exporters import CsvItemExporter

class ExpressPipeline(object):

  def __init__(self):
    self.files = {}

  @classmethod
  def from_crawler(cls, crawler):
    pipeline = cls()
    crawler.signals.connect(pipeline.spider_opened, signals.spider_opened)
    crawler.signals.connect(pipeline.spider_closed, signals.spider_closed)
    return pipeline

  def spider_opened(self, spider):
    file = open('%s_items.csv' % spider.name, 'w+b')
    self.files[spider] = file
    self.exporter = CsvItemExporter(file)
    self.exporter.fields_to_export = ['text', 'name', 'org', 'loc', 'keyword', 'sources', 'hdfcpresent', 'date']
    self.exporter.start_exporting()

  def spider_closed(self, spider):
    self.exporter.finish_exporting()
    file = self.files.pop(spider)
    file.close()

  def process_item(self, item, spider):
    self.exporter.export_item(item)
    return item

Overwriting /content/express/express/pipelines.py


In [5]:
%%writefile /content/express/express/settings.py
# Scrapy settings for news project
#
# For simplicity, this file contains only settings considered important or
# commonly used. You can find more settings consulting the documentation:
#
#     https://docs.scrapy.org/en/latest/topics/settings.html
#     https://docs.scrapy.org/en/latest/topics/downloader-middleware.html
#     https://docs.scrapy.org/en/latest/topics/spider-middleware.html

BOT_NAME = 'express'

SPIDER_MODULES = ['express.spiders']
NEWSPIDER_MODULE = 'express.spiders'


# Crawl responsibly by identifying yourself (and your website) on the user-agent
#USER_AGENT = 'news (+http://www.yourdomain.com)'

# Obey robots.txt rules
ROBOTSTXT_OBEY = True

# Configure maximum concurrent requests performed by Scrapy (default: 16)
#CONCURRENT_REQUESTS = 32

# Configure a delay for requests for the same website (default: 0)
# See https://docs.scrapy.org/en/latest/topics/settings.html#download-delay
# See also autothrottle settings and docs
#DOWNLOAD_DELAY = 3
# The download delay setting will honor only one of:
#CONCURRENT_REQUESTS_PER_DOMAIN = 16
#CONCURRENT_REQUESTS_PER_IP = 16

# Disable cookies (enabled by default)
#COOKIES_ENABLED = False

# Disable Telnet Console (enabled by default)
#TELNETCONSOLE_ENABLED = False

# Override the default request headers:
#DEFAULT_REQUEST_HEADERS = {
#   'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
#   'Accept-Language': 'en',
#}

# Enable or disable spider middlewares
# See https://docs.scrapy.org/en/latest/topics/spider-middleware.html
#SPIDER_MIDDLEWARES = {
#    'news.middlewares.NewsSpiderMiddleware': 543,
#}

# Enable or disable downloader middlewares
# See https://docs.scrapy.org/en/latest/topics/downloader-middleware.html
#DOWNLOADER_MIDDLEWARES = {
#    'news.middlewares.NewsDownloaderMiddleware': 543,
#}

# Enable or disable extensions
# See https://docs.scrapy.org/en/latest/topics/extensions.html
#EXTENSIONS = {
#    'scrapy.extensions.telnet.TelnetConsole': None,
#}

# Configure item pipelines
# See https://docs.scrapy.org/en/latest/topics/item-pipeline.html
ITEM_PIPELINES = {
   'express.pipelines.ExpressPipeline': 300,
}

# Enable and configure the AutoThrottle extension (disabled by default)
# See https://docs.scrapy.org/en/latest/topics/autothrottle.html
#AUTOTHROTTLE_ENABLED = True
# The initial download delay
#AUTOTHROTTLE_START_DELAY = 5
# The maximum download delay to be set in case of high latencies
#AUTOTHROTTLE_MAX_DELAY = 60
# The average number of requests Scrapy should be sending in parallel to
# each remote server
#AUTOTHROTTLE_TARGET_CONCURRENCY = 1.0
# Enable showing throttling stats for every response received:
#AUTOTHROTTLE_DEBUG = False

# Enable and configure HTTP caching (disabled by default)
# See https://docs.scrapy.org/en/latest/topics/downloader-middleware.html#httpcache-middleware-settings
#HTTPCACHE_ENABLED = True
#HTTPCACHE_EXPIRATION_SECS = 0
#HTTPCACHE_DIR = 'httpcache'
#HTTPCACHE_IGNORE_HTTP_CODES = []
#HTTPCACHE_STORAGE = 'scrapy.extensions.httpcache.FilesystemCacheStorage'


Overwriting /content/express/express/settings.py


In [6]:
%cd /content/
!ls express/express/spiders

/content
__init__.py  news_spider.py  __pycache__


In [7]:
%%writefile express/express/spiders/news_spider.py
import os
import csv
from dateutil.parser import parse


import scrapy
from bs4 import BeautifulSoup
from express.items import ExpressItem
import spacy

nlp_Name = spacy.load("en_core_web_trf") # spacy.load(OUTPUT1)

def get_urls_from_csv():
    with open('websites.csv', 'rbU') as csv_file:
        data = csv.reader(csv_file)
        scrapurls = []
        for row in data:
            scrapurls.append(row)
        return scrapurls

class ExpressSpider(scrapy.Spider):
    name = "express"
    
    start_urls = [
                'https://indianexpress.com/article/cities/mumbai/complainant-activist-moves-bombay-hc-seeking-transfer-of-probe-from-eow-to-cbi-7206619/',
                'https://indianexpress.com/article/cities/kolkata/mamata-banerjee-jagannath-puja-bengal-elections-7205809/',
                'https://indianexpress.com/article/cities/mumbai/arthur-road-jail-special-cell-ready-nirav-modi-7205674/',
                'https://indianexpress.com/article/india/from-flashy-diamantaire-to-fugitive-nirav-modis-long-road-to-extradition-7204773/',
                'https://indianexpress.com/article/india/nirav-modi-case-timeline-7204705/',
                ]
    
    # start_urls = [url+keyword for url in start_urls for keyword in ['black money',
    #                                                                 'money laundering', 
    #                                                                 'money launder', 
    #                                                                 'lauder the money', 
    #                                                                 'money-mule', 
    #                                                                 'money mule', 
    #                                                                 'Hawala', 
    #                                                                 'drug-trafficking', 
    #                                                                 'drug trafficking', 
    #                                                                 'terror', 
    #                                                                 'terror financing'
    # ]]

    def parse(self, response):
      # text data of entire webpage
      soup = BeautifulSoup(response.text, 'html.parser') # parse
      # extract body from it
      tag = soup.body # scrape.parse_soup
      
      # save data in profile
      profile = ExpressItem()
      profile['text'] = ''
      profile['name'] = ''
      profile['org'] = ''
      profile['date'] = ''
      profile['loc'] = ''
      profile['keyword'] = ''
      profile['hdfcpresent'] = 'No'
      # profile = {'text': '', 'name': '', 'org': '', 'loc': '', 'facebook_link': '', 'twitter_link': '', 'other_weblinks': '', 'image_link': '', 'keyword': '', 'sources': '', 'hdfcpresent': 'No', 'date': ''}

      fp_name = [
                 'Swasth',
                 'Corona',
                 'Femina',
                 'ETimes',
                 'Grazia',
                 'RTI Bill',
                 'The Economic Times',
                 'Hot on the Web',
                 'Corona cases in India',
                 'EMail',
                 'Jharkhand',
                 'Gujarat',
                 'Assam',
                 'Haryana',
                 'Mizoram',
                 'Nagaland',
                 'Odisha',
                 'Sikkim',
                 'Tripura',
                 'Dadra Nagar Haveli',
                 'Daman Diu',
                 'Lakshadweep',
                 ]

      fp_org = [
                'NDTV',
                ''

                 ]

      fp_loc = [


                 ]

      fp_date = [


                 ]
      
      keywords = [
                  'fraud',
                  'scam',
                  'money laundering',
                  'money laundering', 
                  'money launder', 
                  'lauder the money', 
                  'money-mule', 
                  'money mule', 
                  'hawala', 
                  'drug-trafficking', 
                  'drug trafficking', 
                  'terror', 
                  'terror financing',
                  'drug traffickers']

      # iterate through each string
      for string in tag.stripped_strings:

        for keyword in keywords:
          if keyword in str(string).lower():
            if keyword not in profile['keyword']:
              profile['keyword'] += keyword + ', '
        
        if 'HDFC' in str(string).lower():
          profile['hdfcpresent'] = 'Yes'


        if len(string) > 540:
          continue
        
        # name extration through spacy model
        doc = nlp_Name(str(string))
        
        # iterate through each entity present
        for count,ent in enumerate(doc.ents):
          # print(ent.label_, ':', ent.text)
          
          # find persons in text
          if ent.label_ == 'PERSON':
            
            # remove name if present in false positives
            if str(string) not in fp_name and (ent.text not in profile['name']):
              # print(str(string))
              profile['text'] = str(string)
              profile['name'] += ent.text + ', '
            else:
              # print('Persons entity:', ent.text, ':', ent.label_)
              pass
          
          # find persons in text
          elif ent.label_ == 'ORG':
            
            # remove name if present in false positives
            if str(string) not in fp_org and (ent.text not in profile['org']):
              profile['org'] += ent.text + ', '
            else:
              # print('Persons entity:', ent.text, ':', ent.label_)
              pass

          # find persons in text
          elif ent.label_ == 'GPE':
            
            # remove name if present in false positives
            if str(string) not in fp_loc and (ent.text not in profile['loc']):
              profile['loc'] += ent.text + ', '

          # find persons in text
          elif ent.label_ == 'DATE':
            
            # remove name if present in false positives
            if str(string) not in fp_date and (ent.text not in profile['date']) and (not (ent.text).isalpha()) and (not (ent.text).isdigit())and (not profile['date']):
              try:
                profile['date'] += str(parse(ent.text)) + ', '
              except Exception as e:
                print(e)
                profile['date'] = ''
                pass
          
      
      profile['sources'] = response.url

      yield profile

    # def parse(self, response):
    #   # text data of entire webpage
    #   soup = BeautifulSoup(response.text, 'html.parser') # parse
    #   # extract body from it
    #   soup = soup.body # scrape.parse_soup

    #   all_links = soup.findAll("a", href=True)

    #   for link in all_links:
    #     news = HinduItem()
    #     if link.string and (response.url.split('/')[2] in link['href']):
    #       news['headline'] = link.string
    #       news['url'] = response.urljoin(link['href'])

    #       yield news

      # all_links = soup.findAll("a", {'target': '_blank'})

      # for link in all_links:
      #   news = NewsItem()
      #   news['headline'] = link.find('h2')
      #   if news['headline']:
      #     news['headline'] = news['headline'].string
      #     news['url'] = response.urljoin(link['href'])

      #     yield news

Overwriting express/express/spiders/news_spider.py


In [8]:
%cd /content/express/
!ls

/content/express
express  scrapy.cfg  websites.csv


In [9]:
%%writefile websites.csv
https://economictimes.indiatimes.com/topic/,
https://timesofindia.indiatimes.com/topic/,
https://www.ndtv.com/search?searchtext=

Overwriting websites.csv


In [10]:
!scrapy crawl express

2021-02-27 05:45:29.526393: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-02-27 05:45:43 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: express)
2021-02-27 05:45:43 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.10 (default, Feb 20 2021, 21:17:23) - [GCC 7.5.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1j  16 Feb 2021), cryptography 3.4.6, Platform Linux-4.19.112+-x86_64-with-Ubuntu-18.04-bionic
2021-02-27 05:45:43 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-02-27 05:45:43 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'express',
 'NEWSPIDER_MODULE': 'express.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['express.spiders']}
2021-02-27 05:45:43 [scrapy.extensions.telnet] INFO: Telnet Password: 28b691a2709ff117
2021-02-27 05:45:43 [scrapy.middleware] INFO: Enabled ext

In [11]:
%cd /content/express/
!ls

/content/express
express  express_items.csv  scrapy.cfg	websites.csv


In [12]:
import pandas as pd

In [13]:
df = pd.read_csv('express_items.csv')

In [14]:
df.head()

,text,name,org,loc,keyword,sources,hdfcpresent,date
0,The petitioners have filed a protest petition ...,"Hardik Patel, Deep Sidhu, Surinder Mohan Arora...","BJP, Congress, UN, Bombay HC, EOW, CBI, MSCB, ...","Bangla, Hindi, India, Sri Lanka, Gujarat, Delh...","fraud, scam,",https://indianexpress.com/article/cities/mumba...,No,"2021-02-27 00:00:00,"
1,Has ECI decided Bengal poll schedule on Modi-S...,"Hardik Patel, Deep Sidhu, Abhishek Banerjee, M...","BJP, Congress, UN, TMC, the Election Commissio...","Bangla, Hindi, India, Sri Lanka, Gujarat, Delh...","scam,",https://indianexpress.com/article/cities/kolka...,No,"2021-02-27 00:00:00,"
2,Uddhav slams Centre for not granting Marathi l...,"Hardik Patel, Deep Sidhu, Nirav Modi, Uddhav,","BJP, Congress, UN, Punjab National Bank, PNB, ...","Bangla, Hindi, India, Sri Lanka, Gujarat, Delh...","fraud, scam, money laundering,",https://indianexpress.com/article/cities/mumba...,No,"2021-02-27 00:00:00,"
3,"In Wang-Jaishankar talk: a new hotline, paths ...","Hardik Patel, Deep Sidhu, Nirav Modi, Mehul Ch...","BJP, Congress, UN, Punjab National Bank, PNB, ...","Bangla, Hindi, India, Sri Lanka, Gujarat, Delh...","fraud, scam, money laundering,",https://indianexpress.com/article/india/nirav-...,No,"2021-02-27 00:00:00,"
4,“Coupled with a severe condition of depression...,"Hardik Patel, Deep Sidhu, Nirav Modi's, Nirav ...","BJP, Congress, UN, Punjab National Bank, PNB, ...","Bangla, Hindi, India, Sri Lanka, Gujarat, Delh...","fraud, scam, money laundering,",https://indianexpress.com/article/india/from-f...,No,"2021-02-27 00:00:00,"


In [15]:
import numpy as np

# replace empty string na values
df.replace(to_replace='', value=np.nan, inplace=True)

In [16]:
# check keywords in all the rows
null_keywords = df[df['keyword'].isna()]

In [17]:
null_keywords.index.tolist()

[]

In [ ]:
print(null_keywords.loc[22, 'sources'])

In [ ]:
df.loc[0,'keyword'] = 'money laundering'
df.loc[7,'keyword'] = 'money laundering'
df.loc[9,'keyword'] = 'money laundering'
df.loc[12,'keyword'] = 'money laundering'
df.loc[16,'keyword'] = 'drug trafficking'
df.loc[19,'keyword'] = 'hawala '
df.loc[21,'keyword'] = 'drug trafficking'
df.loc[22,'keyword'] = 'drug trafficking'
df.loc[23,'keyword'] = 'drug trafficking'
df.loc[25,'keyword'] = 'drug trafficking'

In [18]:
df.drop(['text' ], axis=1, inplace=True)

In [19]:
df.head()

,name,org,loc,keyword,sources,hdfcpresent,date
0,"Hardik Patel, Deep Sidhu, Surinder Mohan Arora...","BJP, Congress, UN, Bombay HC, EOW, CBI, MSCB, ...","Bangla, Hindi, India, Sri Lanka, Gujarat, Delh...","fraud, scam,",https://indianexpress.com/article/cities/mumba...,No,"2021-02-27 00:00:00,"
1,"Hardik Patel, Deep Sidhu, Abhishek Banerjee, M...","BJP, Congress, UN, TMC, the Election Commissio...","Bangla, Hindi, India, Sri Lanka, Gujarat, Delh...","scam,",https://indianexpress.com/article/cities/kolka...,No,"2021-02-27 00:00:00,"
2,"Hardik Patel, Deep Sidhu, Nirav Modi, Uddhav,","BJP, Congress, UN, Punjab National Bank, PNB, ...","Bangla, Hindi, India, Sri Lanka, Gujarat, Delh...","fraud, scam, money laundering,",https://indianexpress.com/article/cities/mumba...,No,"2021-02-27 00:00:00,"
3,"Hardik Patel, Deep Sidhu, Nirav Modi, Mehul Ch...","BJP, Congress, UN, Punjab National Bank, PNB, ...","Bangla, Hindi, India, Sri Lanka, Gujarat, Delh...","fraud, scam, money laundering,",https://indianexpress.com/article/india/nirav-...,No,"2021-02-27 00:00:00,"
4,"Hardik Patel, Deep Sidhu, Nirav Modi's, Nirav ...","BJP, Congress, UN, Punjab National Bank, PNB, ...","Bangla, Hindi, India, Sri Lanka, Gujarat, Delh...","fraud, scam, money laundering,",https://indianexpress.com/article/india/from-f...,No,"2021-02-27 00:00:00,"


In [20]:
df.columns

Index(['name', 'org', 'loc', 'keyword', 'sources', 'hdfcpresent', 'date'], dtype='object')

In [21]:
df.columns = ['Person_Name_mentioned_in_the_news', 
              'Organization_Name_mentioned_in_the_news', 
              'City_State_mentioned_under_the_news', 
              'Key_word_Used_for_identify_the_article', 
              'Web_link_of_news',
              'HDFC_Bank_Name_under_News_Article', 
              'Article_Date']

In [22]:
df.head()

,Person Name mentioned in the news,Organization Name mentioned in the news,City/ State mentioned under the news,Key word Used for identify the article,Web link of news,HDFC Bank Name under News / Article,Article Date
0,"Hardik Patel, Deep Sidhu, Surinder Mohan Arora...","BJP, Congress, UN, Bombay HC, EOW, CBI, MSCB, ...","Bangla, Hindi, India, Sri Lanka, Gujarat, Delh...","fraud, scam,",https://indianexpress.com/article/cities/mumba...,No,"2021-02-27 00:00:00,"
1,"Hardik Patel, Deep Sidhu, Abhishek Banerjee, M...","BJP, Congress, UN, TMC, the Election Commissio...","Bangla, Hindi, India, Sri Lanka, Gujarat, Delh...","scam,",https://indianexpress.com/article/cities/kolka...,No,"2021-02-27 00:00:00,"
2,"Hardik Patel, Deep Sidhu, Nirav Modi, Uddhav,","BJP, Congress, UN, Punjab National Bank, PNB, ...","Bangla, Hindi, India, Sri Lanka, Gujarat, Delh...","fraud, scam, money laundering,",https://indianexpress.com/article/cities/mumba...,No,"2021-02-27 00:00:00,"
3,"Hardik Patel, Deep Sidhu, Nirav Modi, Mehul Ch...","BJP, Congress, UN, Punjab National Bank, PNB, ...","Bangla, Hindi, India, Sri Lanka, Gujarat, Delh...","fraud, scam, money laundering,",https://indianexpress.com/article/india/nirav-...,No,"2021-02-27 00:00:00,"
4,"Hardik Patel, Deep Sidhu, Nirav Modi's, Nirav ...","BJP, Congress, UN, Punjab National Bank, PNB, ...","Bangla, Hindi, India, Sri Lanka, Gujarat, Delh...","fraud, scam, money laundering,",https://indianexpress.com/article/india/from-f...,No,"2021-02-27 00:00:00,"


In [23]:
df['Source_of_Info'] = 'News Paper'

In [24]:
df['Source_Name'] = 'The Indian Express'

In [25]:
import numpy as np

df['City_of_News_Paper'] = np.nan

In [26]:
# strip ending comma, spaces
df['Person Name mentioned in the news'] = df['Person Name mentioned in the news'].apply(lambda x: x.strip(', '))
df['Organization Name mentioned in the news'] = df['Organization Name mentioned in the news'].apply(lambda x: x.strip(', '))
df['City/ State mentioned under the news'] = df['City/ State mentioned under the news'].apply(lambda x: x.strip(', '))
df['Key word Used for identify the article'] = df['Key word Used for identify the article'].apply(lambda x: x.strip(', '))
df['HDFC Bank Name under News / Article'] = df['HDFC Bank Name under News / Article'].apply(lambda x: x.strip(', '))
df['Article Date'] = df['Article Date'].apply(lambda x: x.strip(', '))

In [27]:
df['Person Name mentioned in the news'][0]

'Hardik Patel, Deep Sidhu, Surinder Mohan Arora, Satish Talekar, Ajit Pawar, Ajit Deshmukh, Uddhav, S S Shinde, Manish Pitale'

In [34]:
df['Article Date'][0]

'2021-02-27 00:00:00'

In [28]:
df.columns

Index(['Person Name mentioned in the news',
       'Organization Name mentioned in the news',
       'City/ State mentioned under the news',
       'Key word Used for identify the article', 'Web link of news',
       'HDFC Bank Name under News / Article', 'Article Date', 'Source of Info',
       'Source Name', 'City of News Paper'],
      dtype='object')

In [29]:
df.shape

(5, 10)

In [30]:
df.to_json('indian_express.json', orient='records', lines=True)